# MSE & Mean Pearson R Inspection

    Comparing the Performance of the different models yieleded almost the same MSE for almost all models and some models get a correlation score of "NaN". We inspect why.

In [1]:
import sys
sys.path.append('../')
sys.path.append('../../')

In [2]:
from utils.model_analysis import * 
import regex as re
import pandas as pd
import torch.nn.functional as F

# Correlation

### Inspecting Predictions from Resnet 18 - Complex Head - PCA 100

In [10]:
#Change as needed
paths = ['/Users/emilykruger/Documents/GitHub/aml_project_2023/hpc/utils/trained_models/03-01-24/resnet18_LR0.00015_SAMPLES_all_SIMPLEHEAD_False_EPOCHS50_BATCHSIZE_128_TIME_2024-01-02_16:34:13.pt']
                   
backbone = 'resnet18'
subject_ids = list(range(1,8+1))

In [11]:
for checkpoint_path in paths:
    # Use the regex to extract the part after "trained_models/"
    pattern = re.compile(r'_models/(.*)')
    match = pattern.search(checkpoint_path)
    model_name = match.group(1)
    corr_list = []
    mse_list = []
    pred_list = []
    #loop over all subjects to calculate MNNPC per subject
    for subject_id in subject_ids:
        #load images and original activations 
        brain, image_paths = load_subject_data(subject_id)

        #pick image and activation
        images = []
        for i in range(len(image_paths)):
            image = np.load(image_paths[i])
            image = preprocess(image)
            images.append(image)
        activation = torch.Tensor(brain)

        output_size = 100

        # Check if GPU is available and if not, use CPU
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        #Load in model & Create feature extractor
        trained_model_state_dict = torch.load(checkpoint_path, map_location = device)
        feature_extractor = torch.hub.load('../utils', backbone, source = 'local')

        trained_model = ResNet1HeadID(output_size = output_size, feature_extractor= feature_extractor, simple_head=False).eval()
        trained_model.load_state_dict(trained_model_state_dict["model_state_dict"])
        trained_model_backbone = trained_model.feature_extractor

        #Inverse transform of preds with frozen PCA models
        pca = get_pca_model(subject_id)
        
        #use loaded model from beginning and pass image through
        preds = np.zeros(activation.shape)
        for i, img in enumerate(images):
            pred = trained_model(img.unsqueeze(0))
            inversed_pred = pca.inverse_transform(pred.detach().numpy()).flatten()
            preds[i,:] = inversed_pred

        predictions = torch.Tensor(preds)
        
        #calculate normalized corr score
        corr = norm_pearson(gt_activation=activation, pred_activation=predictions)

        mse_score= F.mse_loss(predictions, activation)
        #append to list
        corr_list.append(corr)
        mse_list.append(mse_score)
        pred_list.append(predictions)

        print(f'Subject {subject_id} done.')

Shape of pca_brain:  (984, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 

100%|██████████| 39548/39548 [00:01<00:00, 36969.46it/s]


Subject 1 done.
Shape of pca_brain:  (984, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runni

100%|██████████| 39548/39548 [00:01<00:00, 24069.42it/s]


Subject 2 done.
Shape of pca_brain:  (908, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runni

...

Subject 6 done.
Shape of pca_brain:  (984, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runni

100%|██████████| 39548/39548 [00:00<00:00, 45017.68it/s]


Subject 7 done.
Shape of pca_brain:  (877, 39511)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runni

100%|██████████| 39511/39511 [00:00<00:00, 53223.92it/s]

Subject 8 done.


In [12]:
print(f'MSE list: {mse_list}\nCorr-List: {corr_list}')

MSE list: [tensor(0.5035), tensor(0.5154), tensor(0.5735), tensor(0.5886), tensor(0.5214), tensor(0.5889), tensor(0.4518), tensor(0.5705)]
Corr-List: [nan, nan, nan, nan, nan, nan, nan, nan]


In [18]:
pred_list[0][:,1]

tensor([0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061,
        0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061,
        0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061,
        0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061,
        0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061,
        0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061,
        0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061,
        0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061,
        0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061,
        0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061,
        0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061,
        0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061,
        0.0061, 0.0061, 0.0061, 0.0061, 

In [20]:
pred_list[3][:,5]

tensor([0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251,
        0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251,
        0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251,
        0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251,
        0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251,
        0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251,
        0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251,
        0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251,
        0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251,
        0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251,
        0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251,
        0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251, 0.0251,
        0.0251, 0.0251, 0.0251, 0.0251, 

    Trying different indeces for subject and voxel all yield the same result: Our model seems to predict the exact same activation values for all images. Let's try another model

### Inspecting Predictions from Resnet101 - Complex Head - No PCA

In [21]:
#Change as needed
paths = ['/Users/emilykruger/Documents/GitHub/aml_project_2023/hpc/utils/trained_models/03-01-24/resnet101_LR0.0003_SAMPLES_all_SIMPLEHEAD_False_EPOCHS50_BATCHSIZE_64_TIME_2024-01-02_23:24:48.pt']               
backbone = 'resnet101'
subject_ids = list(range(1,8+1))

In [22]:
for checkpoint_path in paths:
    # Use the regex to extract the part after "trained_models/"
    pattern = re.compile(r'03-01-24/(.*)')
    match = pattern.search(checkpoint_path)
    model_name = match.group(1) + 'NO_PCA'
    corr_list = []
    mse_list = []
    pred_list = []
    #loop over all subjects to calculate performance metrics per subject
    for subject_id in subject_ids:
        #load images and original activations 
        brain, image_paths = load_subject_data(subject_id)

        #pick image and activation
        images = []
        for i in range(len(image_paths)):
            image = np.load(image_paths[i])
            image = preprocess(image)
            images.append(image)
            
        activation = torch.Tensor(brain[:,:39198])

        output_size = activation.shape[1]

        # Check if GPU is available and if not, use CPU
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        #Load in model & Create feature extractor
        trained_model_state_dict = torch.load(checkpoint_path, map_location = device)
        feature_extractor = torch.hub.load('../utils', backbone, source = 'local')

        trained_model = ResNet1HeadID(output_size = output_size, feature_extractor= feature_extractor, simple_head = False).eval()
        trained_model.load_state_dict(trained_model_state_dict["model_state_dict"])
        trained_model_backbone = trained_model.feature_extractor

        #use loaded model from beginning and pass image through
        preds = np.zeros(activation.shape)
        for i, img in enumerate(images):
            pred = trained_model(img.unsqueeze(0))
            preds[i,:] = pred.detach().numpy()

        predictions = torch.Tensor(preds)

        #calculate normalized corr score
        corr = norm_pearson(gt_activation=activation, pred_activation=predictions)

        mse_score= F.mse_loss(predictions, activation)
        #append to list
        pred_list.append(predictions)
        corr_list.append(corr)
        mse_list.append(mse_score)

        print(f'Subject {subject_id} done.')

Shape of pca_brain:  (984, 39548)


/Users/emilykruger/Documents/GitHub/aml_project_2023/aml_project_2023/aml_venv/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

  0%|          | 0/39198 [00:00<?, ?it/s]/Users/emilykruger/Documents/GitHub/aml_project_2023/aml_project_2023/aml_venv/lib/python3.11/site-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
100%|██████████| 39198/39198 [00:00<00:00, 61174.21it/s]


Subject 1 done.
Shape of pca_brain:  (984, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0

...

Subject 6 done.
Shape of pca_brain:  (984, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0

100%|██████████| 39198/39198 [00:00<00:00, 58217.30it/s]


Subject 7 done.
Shape of pca_brain:  (877, 39511)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0

100%|██████████| 39198/39198 [00:00<00:00, 58264.05it/s]


Subject 8 done.


In [23]:
print(f'MSE list: {mse_list}\nCorr-List: {corr_list}')

MSE list: [tensor(0.5035), tensor(0.5154), tensor(0.5733), tensor(0.5882), tensor(0.5214), tensor(0.5888), tensor(0.4517), tensor(0.5705)]
Corr-List: [nan, nan, nan, nan, nan, nan, nan, nan]


In [24]:
pred_list[7][:,14556]

tensor([-0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004,
        -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004,
        -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004,
        -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004,
        -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004,
        -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004,
        -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004,
        -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004,
        -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004,
        -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004,
        -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004,
        -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004, -0.0004,
        -0.0004, -0.0004, -0.0004, -0.00

    This model too seems to predict the exact same activation values for all images. This explains the NaN values in the correlation score: If there is no variance at all in the prediciton, we cannot find a correlation.

# MSE

    Inspecting the predictons from the two models above also revealed, that all predicted values are very close to 0. What happens if we only predicted 0?

In [9]:
#load sample data for subject 1
brain, image_paths = load_subject_data(subject = 1)
activation = torch.Tensor(brain)

#create dummy prediction of only 0
dummy = torch.zeros(activation.shape)

#calculate mse
mse_score = F.mse_loss(dummy, activation)
print(mse_score)

Shape of pca_brain:  (984, 39548)
tensor(0.5035)


    Predicting only 0 yields a MSE of 0.5, which is very close to the MSE of our models.